In [170]:
# Imports
import pandas as pd
import numpy as np
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import hvplot.pandas
from sportsipy.nfl.boxscore import Boxscores, Boxscore
from sportsipy.nfl.schedule import Schedule, Game
from sportsipy.nfl.teams import Teams
pd.set_option('display.max_columns', None)

In [171]:
# Import dataframes
test_df = pd.read_csv(Path('Resources/test_df.csv'), index_col='Unnamed: 0')
train_df = pd.read_csv(Path('Resources/train_df.csv'), index_col='Unnamed: 0')
overall_df = pd.read_csv(Path('Resources/overall_df.csv'), index_col='Unnamed: 0')

In [172]:
pred_week = 13
current_week = pred_week
train_df = overall_df[overall_df['week'] < pred_week]
test_df = overall_df[overall_df['week'] == pred_week]

In [173]:
train_df_info = train_df
test_df_info = test_df

display(train_df.tail())
display(test_df.tail())

,away_name,away_abbr,home_name,home_abbr,week,win_perc_dif,first_downs_dif,fumbles_dif,interceptions_dif,net_pass_yards_dif,pass_attempts_dif,pass_completions_dif,pass_touchdowns_dif,pass_yards_dif,penalties_dif,points_dif,rush_attempts_dif,rush_touchdowns_dif,rush_yards_dif,time_of_possession_dif,times_sacked_dif,total_yards_dif,turnovers_dif,yards_from_penalties_dif,yards_lost_from_sacks_dif,fourth_down_perc_dif,third_down_perc_dif,result,elo_dif,qb_dif
26,New York Jets,nyj,Denver Broncos,den,3,-1.0,-6.0,0.0,2.5,-83.0,0.0,-7.5,-1.0,-62.0,-1.0,-15.0,-5.0,-0.5,-32.0,-396.0,2.5,-115.0,2.0,-22.5,21.0,-0.600000,0.000000,0.0,-151.570462,-101.118119
27,Tampa Bay Buccaneers,tam,Los Angeles Rams,ram,3,0.0,3.0,1.0,0.5,28.0,15.0,8.5,2.0,28.0,7.0,9.0,-8.5,-1.0,-20.5,-36.0,0.5,7.5,1.5,69.0,0.0,0.000000,-0.058696,0.0,65.200570,52.581498
28,Seattle Seahawks,sea,Minnesota Vikings,min,3,0.5,-5.5,-1.0,0.0,-8.5,-14.0,-9.5,0.5,-4.5,1.0,0.5,-2.0,0.5,-13.5,-491.0,1.0,-22.0,0.0,10.5,4.0,-1.000000,0.073260,0.0,100.073039,-22.712921
29,Green Bay Packers,gnb,San Francisco 49ers,sfo,3,-0.5,-2.5,-1.5,1.0,-43.5,3.0,1.0,0.5,-26.0,-1.0,-10.0,-10.0,-1.0,-54.5,-78.5,2.0,-98.0,0.5,-24.0,17.5,-0.333333,-0.075515,1.0,68.790231,91.991421
30,Philadelphia Eagles,phi,Dallas Cowboys,dal,3,0.0,-6.5,0.5,-1.0,-87.0,-13.0,-13.0,0.0,-93.0,3.0,-3.0,5.5,0.0,33.0,-221.0,0.0,-54.0,-1.0,7.5,-6.0,0.000000,-0.115556,0.0,-17.083209,-84.813484


,away_name,away_abbr,home_name,home_abbr,week,win_perc_dif,first_downs_dif,fumbles_dif,interceptions_dif,net_pass_yards_dif,pass_attempts_dif,pass_completions_dif,pass_touchdowns_dif,pass_yards_dif,penalties_dif,points_dif,rush_attempts_dif,rush_touchdowns_dif,rush_yards_dif,time_of_possession_dif,times_sacked_dif,total_yards_dif,turnovers_dif,yards_from_penalties_dif,yards_lost_from_sacks_dif,fourth_down_perc_dif,third_down_perc_dif,result,elo_dif,qb_dif
42,Arizona Cardinals,crd,Los Angeles Rams,ram,4,0.000000,1.666667,0.000000,1.000000,18.000000,2.666667,4.000000,-0.666667,21.000000,4.333333,2.666667,2.666667,1.000000,26.333333,294.333333,0.666667,44.333333,0.666667,34.000000,3.000000,0.500000,-0.188018,1.0,-76.203227,49.629399
43,Pittsburgh Steelers,pit,Green Bay Packers,gnb,4,-0.333333,-2.000000,0.333333,0.333333,33.333333,11.666667,6.000000,-1.000000,28.000000,1.000000,-6.000000,-7.000000,-0.333333,-26.666667,-24.666667,0.666667,6.666667,0.000000,-10.000000,-5.333333,-0.500000,0.051938,0.0,-127.686964,-51.880670
44,Baltimore Ravens,rav,Denver Broncos,den,4,-0.333333,-1.000000,1.333333,1.000000,-20.666667,-2.666667,-6.666667,-0.333333,-22.000000,-0.666667,2.000000,0.666667,0.666667,58.333333,-282.333333,0.333333,37.666667,1.000000,-2.333333,-1.333333,-0.250000,-0.048321,1.0,83.742218,31.777967
45,Tampa Bay Buccaneers,tam,New England Patriots,nwe,4,0.333333,5.000000,0.333333,-0.333333,123.666667,7.000000,5.333333,2.666667,116.666667,3.000000,16.333333,-7.666667,0.000000,-35.333333,-198.333333,0.000000,88.333333,0.000000,20.666667,-7.000000,-0.166667,-0.030142,1.0,152.458321,138.692264
46,Las Vegas Raiders,rai,Los Angeles Chargers,sdg,4,0.333333,1.333333,0.666667,-0.333333,72.333333,3.333333,0.000000,0.000000,82.333333,-1.000000,7.666667,3.666667,0.666667,4.000000,256.000000,1.000000,76.333333,-0.666667,-12.333333,10.000000,-0.666667,-0.082503,0.0,11.862365,2.556967


In [174]:
# Drop non-numerical columns
train_df = train_df.drop(columns=['away_name','away_abbr','home_name','home_abbr','week'])
test_df = test_df.drop(columns=['away_name','away_abbr','home_name','home_abbr','week'])
display(test_df.head())
display(train_df.tail())

,win_perc_dif,first_downs_dif,fumbles_dif,interceptions_dif,net_pass_yards_dif,pass_attempts_dif,pass_completions_dif,pass_touchdowns_dif,pass_yards_dif,penalties_dif,points_dif,rush_attempts_dif,rush_touchdowns_dif,rush_yards_dif,time_of_possession_dif,times_sacked_dif,total_yards_dif,turnovers_dif,yards_from_penalties_dif,yards_lost_from_sacks_dif,fourth_down_perc_dif,third_down_perc_dif,result,elo_dif,qb_dif
31,-0.666667,2.000000,1.333333,1.000000,22.333333,14.333333,3.666667,-0.666667,9.666667,1.333333,-5.000000,-6.333333,0.000000,-1.333333,-299.333333,-1.666667,21.000000,1.333333,10.333333,-12.666667,0.500000,-0.060606,0.0,-167.281985,-53.649267
32,-0.333333,-6.333333,-0.333333,0.000000,-41.333333,-12.666667,-7.333333,-0.666667,-40.666667,0.666667,-9.000000,-0.666667,-0.333333,-32.666667,-262.000000,0.666667,-74.000000,-0.333333,5.000000,0.666667,-0.400000,-0.044466,0.0,-242.363130,-176.820930
33,0.000000,-1.000000,0.666667,0.000000,3.000000,-9.333333,-7.000000,0.333333,-8.000000,0.000000,6.333333,2.666667,0.333333,14.333333,-280.666667,-1.333333,17.333333,0.333333,-0.333333,-11.000000,0.514286,-0.068627,1.0,-4.518876,-99.570473
34,0.000000,7.333333,0.333333,0.333333,59.000000,4.333333,4.666667,1.333333,53.333333,-6.333333,9.333333,-0.333333,0.000000,-22.333333,163.000000,-0.333333,36.666667,1.333333,-31.333333,-5.666667,0.000000,0.146319,1.0,181.431277,101.201007
35,0.666667,9.666667,1.000000,-1.333333,53.333333,-0.666667,3.000000,0.666667,44.666667,-2.000000,17.000000,12.666667,1.333333,79.333333,365.666667,-1.666667,132.666667,0.000000,-23.666667,-8.666667,0.114286,0.091667,0.0,212.198370,140.617767


,win_perc_dif,first_downs_dif,fumbles_dif,interceptions_dif,net_pass_yards_dif,pass_attempts_dif,pass_completions_dif,pass_touchdowns_dif,pass_yards_dif,penalties_dif,points_dif,rush_attempts_dif,rush_touchdowns_dif,rush_yards_dif,time_of_possession_dif,times_sacked_dif,total_yards_dif,turnovers_dif,yards_from_penalties_dif,yards_lost_from_sacks_dif,fourth_down_perc_dif,third_down_perc_dif,result,elo_dif,qb_dif
26,-1.0,-6.0,0.0,2.5,-83.0,0.0,-7.5,-1.0,-62.0,-1.0,-15.0,-5.0,-0.5,-32.0,-396.0,2.5,-115.0,2.0,-22.5,21.0,-0.600000,0.000000,0.0,-151.570462,-101.118119
27,0.0,3.0,1.0,0.5,28.0,15.0,8.5,2.0,28.0,7.0,9.0,-8.5,-1.0,-20.5,-36.0,0.5,7.5,1.5,69.0,0.0,0.000000,-0.058696,0.0,65.200570,52.581498
28,0.5,-5.5,-1.0,0.0,-8.5,-14.0,-9.5,0.5,-4.5,1.0,0.5,-2.0,0.5,-13.5,-491.0,1.0,-22.0,0.0,10.5,4.0,-1.000000,0.073260,0.0,100.073039,-22.712921
29,-0.5,-2.5,-1.5,1.0,-43.5,3.0,1.0,0.5,-26.0,-1.0,-10.0,-10.0,-1.0,-54.5,-78.5,2.0,-98.0,0.5,-24.0,17.5,-0.333333,-0.075515,1.0,68.790231,91.991421
30,0.0,-6.5,0.5,-1.0,-87.0,-13.0,-13.0,0.0,-93.0,3.0,-3.0,5.5,0.0,33.0,-221.0,0.0,-54.0,-1.0,7.5,-6.0,0.000000,-0.115556,0.0,-17.083209,-84.813484


In [175]:
# Separate Features from labels.
X_train = train_df.drop(columns='result')
y_train = train_df['result']
X_test = test_df.drop(columns='result')
y_test = test_df['result']

display(X_train.head())
display(y_train.head())
display(X_test.head())
display(y_test.head())
X_train.shape

,win_perc_dif,first_downs_dif,fumbles_dif,interceptions_dif,net_pass_yards_dif,pass_attempts_dif,pass_completions_dif,pass_touchdowns_dif,pass_yards_dif,penalties_dif,points_dif,rush_attempts_dif,rush_touchdowns_dif,rush_yards_dif,time_of_possession_dif,times_sacked_dif,total_yards_dif,turnovers_dif,yards_from_penalties_dif,yards_lost_from_sacks_dif,fourth_down_perc_dif,third_down_perc_dif,elo_dif,qb_dif
0,0.0,4.0,-2.0,0.0,121.0,16.0,8.0,0.0,132.0,-4.0,-3.0,-7.0,1.0,-66.0,55.0,1.0,55.0,0.0,-9.0,11.0,-0.666667,0.200000,-50.848658,100.316973
1,1.0,-4.0,-1.0,-1.0,29.0,-13.0,-9.0,2.0,45.0,0.0,13.0,10.0,-1.0,15.0,-23.0,2.0,44.0,-2.0,-20.0,16.0,0.666667,-0.240260,-51.742731,34.616025
2,0.0,4.0,-1.0,0.0,-119.0,-14.0,-9.0,4.0,-128.0,0.0,19.0,12.0,-1.0,60.0,196.0,-1.0,-59.0,-1.0,-2.0,-9.0,1.000000,0.214286,204.627654,54.983874
3,0.0,10.0,0.0,1.0,232.0,24.0,16.0,1.0,247.0,5.0,10.0,0.0,2.0,7.0,445.0,1.0,239.0,1.0,30.0,15.0,0.000000,0.133333,-62.282436,-14.386838
4,0.0,-3.0,0.0,0.0,50.0,-9.0,-9.0,-1.0,55.0,-7.0,9.0,-3.0,1.0,-42.0,-270.0,0.0,8.0,2.0,-10.0,5.0,0.000000,-0.128205,36.874709,-13.318243


0    0.0
1    0.0
2    0.0
3    1.0
4    1.0
Name: result, dtype: float64

,win_perc_dif,first_downs_dif,fumbles_dif,interceptions_dif,net_pass_yards_dif,pass_attempts_dif,pass_completions_dif,pass_touchdowns_dif,pass_yards_dif,penalties_dif,points_dif,rush_attempts_dif,rush_touchdowns_dif,rush_yards_dif,time_of_possession_dif,times_sacked_dif,total_yards_dif,turnovers_dif,yards_from_penalties_dif,yards_lost_from_sacks_dif,fourth_down_perc_dif,third_down_perc_dif,elo_dif,qb_dif
31,-0.666667,2.000000,1.333333,1.000000,22.333333,14.333333,3.666667,-0.666667,9.666667,1.333333,-5.000000,-6.333333,0.000000,-1.333333,-299.333333,-1.666667,21.000000,1.333333,10.333333,-12.666667,0.500000,-0.060606,-167.281985,-53.649267
32,-0.333333,-6.333333,-0.333333,0.000000,-41.333333,-12.666667,-7.333333,-0.666667,-40.666667,0.666667,-9.000000,-0.666667,-0.333333,-32.666667,-262.000000,0.666667,-74.000000,-0.333333,5.000000,0.666667,-0.400000,-0.044466,-242.363130,-176.820930
33,0.000000,-1.000000,0.666667,0.000000,3.000000,-9.333333,-7.000000,0.333333,-8.000000,0.000000,6.333333,2.666667,0.333333,14.333333,-280.666667,-1.333333,17.333333,0.333333,-0.333333,-11.000000,0.514286,-0.068627,-4.518876,-99.570473
34,0.000000,7.333333,0.333333,0.333333,59.000000,4.333333,4.666667,1.333333,53.333333,-6.333333,9.333333,-0.333333,0.000000,-22.333333,163.000000,-0.333333,36.666667,1.333333,-31.333333,-5.666667,0.000000,0.146319,181.431277,101.201007
35,0.666667,9.666667,1.000000,-1.333333,53.333333,-0.666667,3.000000,0.666667,44.666667,-2.000000,17.000000,12.666667,1.333333,79.333333,365.666667,-1.666667,132.666667,0.000000,-23.666667,-8.666667,0.114286,0.091667,212.198370,140.617767


31    0.0
32    0.0
33    1.0
34    1.0
35    0.0
Name: result, dtype: float64

(31, 24)

In [176]:
# Scale the X data
# Instantiate the scaler
X_scaler = StandardScaler()

# Fit the scaler to Train data
X_scaler.fit(X_train)

#Scale the features data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [177]:
# Build the NN model.
number_input_features = len(X_train.columns)

hidden_nodes_layer1 = (number_input_features + 1)//2

hidden_nodes_layer2 = (hidden_nodes_layer1 + 1)//2

hidden_nodes_layer3 = (hidden_nodes_layer2 + 1)//2

display((f'Number of input features: {number_input_features}'))
display((f'Layer 1: {hidden_nodes_layer1}'))
display((f'Layer 2: {hidden_nodes_layer2}'))
display((f'Layer 3: {hidden_nodes_layer3}'))

'Number of input features: 24'

'Layer 1: 12'

'Layer 2: 6'

'Layer 3: 3'

In [178]:
# Create Sequential NN_model
nn = Sequential()

#Add the first hidden layer
nn.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Add the second layer
nn.add(Dense(units=hidden_nodes_layer2, activation="relu"))

# Add the third layer
nn.add(Dense(units=hidden_nodes_layer3, activation="relu"))

# Add the output layer
nn.add(Dense(units=1, activation="sigmoid"))

In [179]:
# Display the Sequential model summary
nn.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_36 (Dense)            (None, 12)                300       
                                                                 
 dense_37 (Dense)            (None, 6)                 78        
                                                                 
 dense_38 (Dense)            (None, 3)                 21        
                                                                 
 dense_39 (Dense)            (None, 1)                 4         
                                                                 
Total params: 403
Trainable params: 403
Non-trainable params: 0
_________________________________________________________________


In [180]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Fit the model
nfl_nn_model = nn.fit(X_train_scaled, y_train, epochs=500)

Epoch 1/500
1/1 [==============================] - 0s 435ms/step - loss: 0.6850 - accuracy: 0.5161
Epoch 2/500
1/1 [==============================] - 0s 2ms/step - loss: 0.6820 - accuracy: 0.5806
Epoch 3/500
1/1 [==============================] - 0s 2ms/step - loss: 0.6797 - accuracy: 0.5484
Epoch 4/500
1/1 [==============================] - 0s 3ms/step - loss: 0.6772 - accuracy: 0.5806
Epoch 5/500
1/1 [==============================] - 0s 3ms/step - loss: 0.6749 - accuracy: 0.5806
Epoch 6/500
1/1 [==============================] - 0s 3ms/step - loss: 0.6730 - accuracy: 0.6452
Epoch 7/500
1/1 [==============================] - 0s 4ms/step - loss: 0.6714 - accuracy: 0.6452
Epoch 8/500
1/1 [==============================] - 0s 4ms/step - loss: 0.6699 - accuracy: 0.6452
Epoch 9/500
1/1 [==============================] - 0s 2ms/step - loss: 0.6685 - accuracy: 0.6452
Epoch 10/500
1/1 [==============================] - 0s 3ms/step - loss: 0.6670 - accuracy: 0.6452
Epoch 11/500
1/1 [=========

In [181]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

1/1 - 0s - loss: 0.6433 - accuracy: 0.6875 - 119ms/epoch - 119ms/step
Loss: 0.6432961225509644, Accuracy: 0.6875


In [182]:
y_pred = nn.predict(X_test_scaled)

y_pred

array([[3.8084495e-01],
       [9.5564275e-05],
       [2.1118727e-01],
       [6.4769435e-01],
       [3.5265109e-01],
       [6.4769435e-01],
       [6.4769435e-01],
       [6.4769435e-01],
       [1.8300089e-01],
       [5.4001743e-01],
       [1.7738494e-01],
       [6.0401529e-01],
       [1.2315452e-01],
       [6.4769435e-01],
       [2.8231621e-01],
       [6.4769435e-01]], dtype=float32)

In [183]:
def get_schedule(year):
    weeks = list(range(1,19))
    schedule_df = pd.DataFrame()
    for w in range(len(weeks)):
        date_string = str(weeks[w]) + '-' + str(year)
        week_scores = Boxscores(weeks[w],year)
        week_games_df = pd.DataFrame()
        for g in range(len(week_scores.games[date_string])):
            game = pd.DataFrame(week_scores.games[date_string][g], index = [0])[['away_name', 'away_abbr','home_name', 'home_abbr','winning_name', 'winning_abbr' ]]
            game['week'] = weeks[w]
            week_games_df = pd.concat([week_games_df,game])
        schedule_df = pd.concat([schedule_df, week_games_df]).reset_index().drop(columns = 'index') 
    return schedule_df 

In [184]:
test_df.head()

,win_perc_dif,first_downs_dif,fumbles_dif,interceptions_dif,net_pass_yards_dif,pass_attempts_dif,pass_completions_dif,pass_touchdowns_dif,pass_yards_dif,penalties_dif,points_dif,rush_attempts_dif,rush_touchdowns_dif,rush_yards_dif,time_of_possession_dif,times_sacked_dif,total_yards_dif,turnovers_dif,yards_from_penalties_dif,yards_lost_from_sacks_dif,fourth_down_perc_dif,third_down_perc_dif,result,elo_dif,qb_dif
31,-0.666667,2.000000,1.333333,1.000000,22.333333,14.333333,3.666667,-0.666667,9.666667,1.333333,-5.000000,-6.333333,0.000000,-1.333333,-299.333333,-1.666667,21.000000,1.333333,10.333333,-12.666667,0.500000,-0.060606,0.0,-167.281985,-53.649267
32,-0.333333,-6.333333,-0.333333,0.000000,-41.333333,-12.666667,-7.333333,-0.666667,-40.666667,0.666667,-9.000000,-0.666667,-0.333333,-32.666667,-262.000000,0.666667,-74.000000,-0.333333,5.000000,0.666667,-0.400000,-0.044466,0.0,-242.363130,-176.820930
33,0.000000,-1.000000,0.666667,0.000000,3.000000,-9.333333,-7.000000,0.333333,-8.000000,0.000000,6.333333,2.666667,0.333333,14.333333,-280.666667,-1.333333,17.333333,0.333333,-0.333333,-11.000000,0.514286,-0.068627,1.0,-4.518876,-99.570473
34,0.000000,7.333333,0.333333,0.333333,59.000000,4.333333,4.666667,1.333333,53.333333,-6.333333,9.333333,-0.333333,0.000000,-22.333333,163.000000,-0.333333,36.666667,1.333333,-31.333333,-5.666667,0.000000,0.146319,1.0,181.431277,101.201007
35,0.666667,9.666667,1.000000,-1.333333,53.333333,-0.666667,3.000000,0.666667,44.666667,-2.000000,17.000000,12.666667,1.333333,79.333333,365.666667,-1.666667,132.666667,0.000000,-23.666667,-8.666667,0.114286,0.091667,0.0,212.198370,140.617767


In [185]:
# Create dataframe to hold prediction results
results_df = test_df_info[['week','away_name','home_name']]
results_df['away_team_win_%'] = y_pred
results_df['home_team_win_%'] = 1 - results_df['away_team_win_%']
results_df['predicted_winner'] = 0

# Loop through results dataframe and add predicted_winner name to column
for index, row in results_df.iterrows():
    if results_df.loc[index,'away_team_win_%'] > results_df.loc[index,'home_team_win_%']:
        results_df.loc[index,'predicted_winner'] = results_df.loc[index,'away_name']
    else:
        results_df.loc[index,'predicted_winner'] = results_df.loc[index,'home_name']

# Set Index to the 'week' column
results_df=results_df.set_index('week')      

# Use get_schedule function to pull dataframe that shows actual weekly winners. Includes all weeks.
weekly_winner = get_schedule('2021')
weekly_winner = weekly_winner.drop(columns=['away_abbr','home_abbr','winning_abbr'])
weekly_winner = weekly_winner.rename(columns={'away_name':'ww_away_name','home_name':'ww_home_name'})

# Slices weekly_winner dataframe based on current week model is predicting.
current_week_wins = weekly_winner.loc[weekly_winner['week'] == current_week]
current_week_wins = current_week_wins.set_index('week')

# Sort both dataframes by away_name
results_df = results_df.sort_values(by='away_name')
current_week_wins = current_week_wins.sort_values(by='ww_away_name')

# Combine results & weekly_winner df's. Drop dupe columns and rename winning_name to 'actual_winner'
actual_results_df = (pd.concat([results_df,current_week_wins],axis=1)).drop(columns=['ww_away_name','ww_home_name']).rename(columns={'winning_name':'actual_winner'})
actual_results_df


C:\Users\14065\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\14065\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\14065\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

,away_name,home_name,away_team_win_%,home_team_win_%,predicted_winner,actual_winner
week,,,,,,
4,Arizona Cardinals,Los Angeles Rams,0.604015,0.395985,Arizona Cardinals,Arizona Cardinals
4,Baltimore Ravens,Denver Broncos,0.647694,0.352306,Baltimore Ravens,Baltimore Ravens
4,Carolina Panthers,Dallas Cowboys,0.183001,0.816999,Dallas Cowboys,Dallas Cowboys
4,Cleveland Browns,Minnesota Vikings,0.647694,0.352306,Cleveland Browns,Cleveland Browns
4,Detroit Lions,Chicago Bears,0.540017,0.459983,Detroit Lions,Chicago Bears
4,Houston Texans,Buffalo Bills,0.000096,0.999904,Buffalo Bills,Buffalo Bills
4,Indianapolis Colts,Miami Dolphins,0.647694,0.352306,Indianapolis Colts,Indianapolis Colts
4,Jacksonville Jaguars,Cincinnati Bengals,0.380845,0.619155,Cincinnati Bengals,Cincinnati Bengals
4,Kansas City Chiefs,Philadelphia Eagles,0.647694,0.352306,Kansas City Chiefs,Kansas City Chiefs


In [186]:
actual_results_df['Accuracy'] = np.where(actual_results_df['predicted_winner'] == actual_results_df['actual_winner'], 1, 0)

actual_results_df = actual_results_df.dropna()

actual_results_df

,away_name,home_name,away_team_win_%,home_team_win_%,predicted_winner,actual_winner,Accuracy
week,,,,,,,
4,Arizona Cardinals,Los Angeles Rams,0.604015,0.395985,Arizona Cardinals,Arizona Cardinals,1
4,Baltimore Ravens,Denver Broncos,0.647694,0.352306,Baltimore Ravens,Baltimore Ravens,1
4,Carolina Panthers,Dallas Cowboys,0.183001,0.816999,Dallas Cowboys,Dallas Cowboys,1
4,Cleveland Browns,Minnesota Vikings,0.647694,0.352306,Cleveland Browns,Cleveland Browns,1
4,Detroit Lions,Chicago Bears,0.540017,0.459983,Detroit Lions,Chicago Bears,0
4,Houston Texans,Buffalo Bills,0.000096,0.999904,Buffalo Bills,Buffalo Bills,1
4,Indianapolis Colts,Miami Dolphins,0.647694,0.352306,Indianapolis Colts,Indianapolis Colts,1
4,Jacksonville Jaguars,Cincinnati Bengals,0.380845,0.619155,Cincinnati Bengals,Cincinnati Bengals,1
4,Kansas City Chiefs,Philadelphia Eagles,0.647694,0.352306,Kansas City Chiefs,Kansas City Chiefs,1


In [187]:
#Week Accuracy 
accuracy = sum(actual_results_df['Accuracy'])/len(actual_results_df)
round(accuracy,2)

0.69

In [188]:
# Export actual_results_df to csv based on the current_week predicted.
file_path = str(f'Resources/week_{current_week}_base_results.csv')
actual_results_df.to_csv(Path(file_path))